In [ ]:
import glob
import numpy as np
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Dense, RNN, LSTM, Flatten, TimeDistributed, LSTMCell, GlobalAveragePooling2D, Input, Dropout, Attention
from tensorflow.keras.layers import RepeatVector, Conv2D, SimpleRNN, GRU, Reshape, ConvLSTM2D, Conv2DTranspose, Conv3DTranspose
from tensorflow.keras import regularizers


#import tensorflow_addons as tfa

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#upload the data

file_inp = '/content/gdrive/MyDrive/ADL_project/Flare_Prediction/Multi_label_classification/train_input.npy'
file_tar = '/content/gdrive/MyDrive/ADL_project/Flare_Prediction/Multi_label_classification/train_target.npy'

train_input = np.load(file_inp)
train_target = np.load(file_tar)

file_inp = '/content/gdrive/MyDrive/ADL_project/Flare_Prediction/Multi_label_classification/val_input.npy'
file_tar = '/content/gdrive/MyDrive/ADL_project/Flare_Prediction/Multi_label_classification/val_target.npy'

val_input = np.load(file_inp)
val_target = np.load(file_tar)

In [ ]:
#example architecture

model = tf.keras.Sequential()
model.add(LSTM(256, input_shape=(10, 40), return_sequences = True))
model.add(LSTM(128, return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(59, return_sequences=True))
model.add(LSTM(59, return_sequences=True))
model.add(LSTM(59))
#model.add(Dropout(rate=0.2))
model.add(RepeatVector(1))
model.add(TimeDistributed(Dense(200, activation = 'relu')))
#model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(100, activation = 'relu')))
model.add(TimeDistributed(Dense(5, activation = 'softmax')))
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(), optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_input, train_target, epochs = 1, shuffle=True, batch_size = 256, validation_data = (val_input, val_target), verbose = False)

In [ ]:
#assessing on the validation set (or do test set if you want to)

pred = model.predict(val_input)

prediction = np.zeros(shape = pred.shape)
for i, p in enumerate(pred):
    j = np.argmax(p[0])
    prediction[i, :, j] = 1


pred = prediction

from sklearn.metrics import classification_report
y_true = np.where(val_target[:, 0, :]==1)[1]
y_pred = np.where(pred[:, 0, :]==1)[1]
target_names = ['No Flare','C', 'B', 'M', 'X']
print(classification_report(y_true, y_pred, target_names=target_names))